In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup

In [2]:
url_construct = "https://fme.discomap.eea.europa.eu/fmedatastreaming/AirQualityDownload/AQData_Extract.fmw?CountryCode={}&CityName={}&Pollutant={}&Year_from=2016&Year_to=2020&Station=&Samplingpoint=&Source=E1a&Output=HTML&UpdateDate=&TimeCoverage=Year"

In [44]:
cities = [{'country': 'GB', 'city': 'London'},
          {'country': 'GB', 'city': 'Greater%20Manchester'},
          {'country': 'PL', 'city': 'Krak%C3%B3w'},
          {'country': 'NL', 'city': 'Amsterdam'},
          {'country': 'IT', 'city': 'Milano'},
          {'country': 'FR', 'city': 'Paris'},
          {'country': 'ES', 'city': 'Madrid'},
          {'country': 'DE', 'city': 'Berlin'},
          {'country': 'DE', 'city': 'Frankfurt%20am%20Main'},
         ]
pollutants = ['5', '6001', '9', '8']

In [16]:
# # Test stuff
# soup = BeautifulSoup(r.content, 'lxml')
# links = []
# for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
#     links.append(link.get('href'))

In [40]:
# master_df = pd.DataFrame()

In [38]:
# for link in links:
#     df = pd.read_csv(link)
#     df['datetime'] = pd.to_datetime(df['DatetimeBegin'], errors='coerce', infer_datetime_format=True) 
#     df['hour'] = df['datetime'].dt.hour
#     df['year'] = df['datetime'].dt.year
#     low = df[df['hour'] == 2]
#     high = df[df['hour'] == 17]
#     dat = {
#         'city': 'London',
#         'site': df['AirQualityStation'].iloc[0],
#         'pollutant': df['AirPollutant'].iloc[0],
#         'year': df['year'].iloc[0],
#         'low': low['Concentration'].mean(),
#         'high': high['Concentration'].mean(),
#         'measurement': df['UnitOfMeasurement'].iloc[0]
#     }
#     master_df = master_df.append(dat, ignore_index=True)

In [ ]:
master_df = pd.DataFrame()
for city in cities:
    for pollutant in pollutants:
        url = url_construct.format(city['country'], city['city'], pollutant)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'lxml')
        links = []
        for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
            links.append(link.get('href'))
        for link in links:
            df = pd.read_csv(link)
            df['datetime'] = pd.to_datetime(df['DatetimeBegin'], errors='coerce', infer_datetime_format=True) 
            df['hour'] = df['datetime'].dt.hour
            df['year'] = df['datetime'].dt.year
            low = df[df['hour'] == 2]
            high = df[df['hour'] == 17]
            midnight = df[df['hour'] == 0]
            dat = {
                'city': city['city'],
                'site': df['AirQualityStation'].iloc[0],
                'pollutant': df['AirPollutant'].iloc[0],
                'year': df['year'].iloc[0],
                'low': low['Concentration'].mean(),
                'high': high['Concentration'].mean(),
                'midnight': midnight['Concentration'].mean(),
                'measurement': df['UnitOfMeasurement'].iloc[0]
            }
            master_df = master_df.append(dat, ignore_index=True)

In [ ]:
master_df